# Day 01 - Data Wrangling

**Author**: Inácio Medeiros

**Date**: 2023-08-10

**Purpose**: Make first contact on CEAPS dataset and perform data wrangling

## Libraries and Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading

In [2]:
df = pd.read_csv('../data/preprocessed/despesa_ceaps_2015.utf8.csv', sep=';')

## Verifying types

In [3]:
df.dtypes

ANO                   int64
MES                   int64
SENADOR              object
TIPO_DESPESA         object
CNPJ_CPF             object
FORNECEDOR           object
DOCUMENTO            object
DATA                 object
DETALHAMENTO         object
VALOR_REEMBOLSADO    object
COD_DOCUMENTO         int64
dtype: object

Based on above list, let's list what must be done:

- Separate CNPJ from CPF
- Convert "DATA" do datetime type
- Evaluate what "DOCUMENTO" means
- Evaluate what "DETALHAMENTO" means
- Convert "VALOR_REEMBOLSADO" to float
- Evaluate what "COD_DOCUMENTO" means

### Separating CNPJ from CPF

In [4]:
df['CNPJ_CPF'].head()

0    05.914.650/0001-66
1    05.914.650/0001-66
2        004.948.028-63
3    05.423.963/0001-11
4    07.170.542/0001-51
Name: CNPJ_CPF, dtype: object

Ok, this shows that actually this field displays either a CNPJ either a CPF. In this sense, we will create a new field called "TIPO_PESSOA" to discern between companies (Pessoa Jurídica) and people (Pessoa Física). The first thing to do is to assert that the strings of these columns has either 14 chars (for CPF) either 18 chars (for CNPJ)

In [5]:
set(df['CNPJ_CPF'].map(lambda v: len(v)))

{14, 18}

In fact they have. Thus, we can use this information for creating our new column:

In [6]:
df['TIPO_PESSOA'] = df['CNPJ_CPF'].map(lambda v: 'PESSOA_FISICA' if len(v) == 14 else 'PESSOA_JURIDICA')

### Convert "DATA" do datetime type

In order to make this run properly, i had to do many manual fixes in the dataset reagrding dates. I believe that for next datasets I will write a function to "remove" or handle more automatically these issues.

In [19]:
df['DATA'] = pd.to_datetime(df['DATA'], dayfirst=True)
df['DATA_DIA'] = df['DATA'].map(lambda v: v.day)
df['DATA_MES'] = df['DATA'].map(lambda v: v.month)
df['DATA_ANO'] = df['DATA'].map(lambda v: v.year)

### Evaluate what "DOCUMENTO" means

This looks like a "code" for the document

In [22]:
df['DOCUMENTO']

0           647777
1        000647776
2           001/15
3        556241490
4            14654
           ...    
26051       7NJ24P
26052       FYNY6G
26053       IGW58E
26054       U7YJHV
26055       7NJ8UP
Name: DOCUMENTO, Length: 26056, dtype: object

### Evaluate what "DETALHAMENTO" means

These are descriptions of the expenses. We could create some categories for it

In [29]:
df['DETALHAMENTO']

0                                                      NaN
1        Despesa com pagamento de energia elétrica para...
2        Despesa com pagamento de aluguel de imóvel par...
3        Despesa com pagamento de telefonia para uso do...
4        Despesa com material de consumo para uso do es...
                               ...                        
26051    Companhia Aérea: AVIANCA, Localizador: 7NJ24P....
26052    Companhia Aérea: AZUL, Localizador: FYNY6G. Pa...
26053    Companhia Aérea: AZUL, Localizador: IGW58E. Pa...
26054    Companhia Aérea: AZUL, Localizador: U7YJHV. Pa...
26055    Companhia Aérea: TAM, Localizador: 7NJ8UP. Pas...
Name: DETALHAMENTO, Length: 26056, dtype: object

### Convert "VALOR_REEMBOLSADO" to float

As we can see below, we need to replace these commas by dots. Here we go

In [30]:
df['VALOR_REEMBOLSADO']

0          52,34
1         168,17
2           5000
3         416,95
4          108,5
          ...   
26051     880,83
26052     717,73
26053     423,35
26054    1540,98
26055     710,54
Name: VALOR_REEMBOLSADO, Length: 26056, dtype: object

In [34]:
df['VALOR_REEMBOLSADO'] = df['VALOR_REEMBOLSADO'].map(lambda v: v.replace(',', '.'))
df['VALOR_REEMBOLSADO'] = df['VALOR_REEMBOLSADO'].map(float)

### Evaluate what "COD_DOCUMENTO" means

This field is really a code for something, differing from "DOCUMENTO"

In [38]:
df['COD_DOCUMENTO']

0        2000857
1        2001140
2        2001144
3        2001141
4        2001139
          ...   
26051    2029890
26052    2029891
26053    2032422
26054    2032243
26055    2029924
Name: COD_DOCUMENTO, Length: 26056, dtype: int64

### Summary

Let's see how are the dtypes now

In [39]:
df.dtypes

ANO                           int64
MES                           int64
SENADOR                      object
TIPO_DESPESA                 object
CNPJ_CPF                     object
FORNECEDOR                   object
DOCUMENTO                    object
DATA                 datetime64[ns]
DETALHAMENTO                 object
VALOR_REEMBOLSADO           float64
COD_DOCUMENTO                 int64
TIPO_PESSOA                  object
DATA_DIA                      int64
DATA_MES                      int64
DATA_ANO                      int64
dtype: object

## Analyzing NULL values

In [52]:
(df['DOCUMENTO'].isna() & df['DETALHAMENTO'].isna()).sum() == df['DOCUMENTO'].isna().sum()

False

In [44]:
df.isna().sum(axis=0)

ANO                     0
MES                     0
SENADOR                 0
TIPO_DESPESA            0
CNPJ_CPF                0
FORNECEDOR              0
DOCUMENTO            4206
DATA                    0
DETALHAMENTO         4053
VALOR_REEMBOLSADO       0
COD_DOCUMENTO           0
TIPO_PESSOA             0
DATA_DIA                0
DATA_MES                0
DATA_ANO                0
dtype: int64

There's a lot of NULL values for DETALHAMENTO and DOCUMENTO. These totals correspond to which percentage of data?

In [48]:
total_number_of_registers = df.shape[0]
percentage_of_null_documents = df['DOCUMENTO'].isna().sum() / total_number_of_registers
percentage_of_null_detalhamento = df['DETALHAMENTO'].isna().sum() / total_number_of_registers
print(f'{percentage_of_null_documents*100:.0f}% of registers has NULL documents')
print(f'{percentage_of_null_detalhamento*100:.0f}% of registers has NULL detalhamentos')

16% of registers has NULL documents
16% of registers has NULL detalhamentos


Shall the intersection of them 100%? Let's see:

In [52]:
(df['DOCUMENTO'].isna() & df['DETALHAMENTO'].isna()).sum() == df['DOCUMENTO'].isna().sum()

False

In [53]:
(df['DOCUMENTO'].isna() & df['DETALHAMENTO'].isna()).sum() == df['DETALHAMENTO'].isna().sum()

False

No, it is not 100%

In [71]:
df[df['SENADOR'].str.contains("FÁTIMA BEZERRA")].groupby('TIPO_DESPESA').agg({'VALOR_REEMBOLSADO': 'sum'})['VALOR_REEMBOLSADO'].sort_values()

TIPO_DESPESA
Serviços de Segurança Privada                                                                                                                                                                        5200.00
Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar                                                               8978.00
Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.      11391.42
Divulgação da atividade parlamentar                                                                                                                                                                 19742.00
Locomoção, hospedagem, alimentação, combustíveis e lubrificantes                                                                                                       

In [58]:
df['DOCUMENTO']

0           647777
1        000647776
2           001/15
3        556241490
4            14654
           ...    
26051       7NJ24P
26052       FYNY6G
26053       IGW58E
26054       U7YJHV
26055       7NJ8UP
Name: DOCUMENTO, Length: 26056, dtype: object

## Saving the work

In [72]:
df.to_csv('../data/preprocessed/despesa_ceaps_2015.data_wrangling.csv', index=False)